In [1]:
myKey = 'GOOGLE_PLACES_API_KEY'

In [2]:
import csv
import googlemaps
import time
from googlemaps.exceptions import ApiError
import pandas as pd

In [3]:


# Initialize the Google Maps client
gmaps = googlemaps.Client(key=myKey)

# Define the cities (Melbourne, London, New York) with their coordinates
cities = [
    {"name": "Colombo", "coordinates": (6.927079, 79.861244)},
    {"name": "Hanoi", "coordinates": (21.028511, 105.804817)},
    {"name": "Dhaka", "coordinates": (23.811056, 90.407608)},
    {"name": "San Francisco", "coordinates": (37.773972, -122.431297)},
    {"name": "Taipei", "coordinates": (25.105497, 121.597366)},
    {"name": "Morocco", "coordinates": (31.625502, -7.988936)},
    {"name": "Melbourne", "coordinates": (-37.8136, 144.9631)},
    {"name": "London", "coordinates": (51.5074, -0.1278)},
    {"name": "New York", "coordinates": (40.7128, -74.0060)},
    {"name": "São Paulo", "coordinates": (-23.5505, -46.6333)},
    {"name": "Tokyo", "coordinates": (35.6762, 139.6503)},
    {"name": "Istanbul", "coordinates": (41.0082, 28.9784)},
    {"name": "Beijing", "coordinates": (39.9042, 116.4074)},
    {"name": "Delhi", "coordinates": (28.6139, 77.2090)},
    {"name": "Cape Town", "coordinates": (-33.9249, 18.4241)}
]

# Setting radius
radius = 25000 # 25 KM
search_terms = ['cafe', 'tea house', 'tea', 'coffee', 'bubble tea', 'coffee shop']

filename = 'cities.csv'
with open(filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['City', 'Place Type', 'Name', 'PlaceID', 'Address', 'Rating', 'Author', 'Review'])

    for city in cities:
        city_name = city['name']
        location = city['coordinates']
        print(f"Searching for places in {city_name}...")

        for term in search_terms:
            try:
 
                places = gmaps.places_nearby(location=location, radius= radius, keyword=term)

                while True:
                    for place in places['results']:
                        name = place['name']
                        placeid = place['place_id']
                        address = place.get('vicinity', 'N/A')
                        rating = place.get('rating', 'N/A')

                        try:
                            place_details = gmaps.place(place_id=placeid)
                            reviews = place_details['result'].get('reviews', [])

                            if reviews:
                                for review in reviews:
                                    author = review.get('author_name', 'N/A')
                                    text = review.get('text', 'N/A')
                                    writer.writerow([city_name, term, name, placeid, address, rating, author, text])
                            else:
                                writer.writerow([city_name, term, name, placeid, address, rating, 'N/A', 'N/A'])

                        except ApiError as e:
                            print(f"Error fetching details for place ID {placeid}: {e}")
                            writer.writerow([city_name, term, name, placeid, address, rating, 'N/A', 'N/A'])

                    if 'next_page_token' in places:
                        time.sleep(2)  # wait 2 seconds before using the next page token
                        places = gmaps.places_nearby(location=location, radius=radius, keyword=term, page_token=places['next_page_token'])
                    else:
                        break
            except ApiError as e:
                print(f"An error occurred in {city_name} for {term}: {e}")

Searching for places in Colombo...
Searching for places in Hanoi...
Searching for places in Dhaka...
Searching for places in San Francisco...
Searching for places in Taipei...
Searching for places in Morocco...
